In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/DataHacks2021/

/content/drive/MyDrive/DataHacks2021


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [4]:
def transform_data(filename, num_days):
    input_data = pd.read_csv(filename, index_col=0)
    input_data = input_data
    sp500 = input_data['SP500']
    lst = [[input_data.iloc[i-num_days:i].values,sp500[i]] for i in range(num_days,len(input_data))]
    final = [[[[item] for sublist in x[0] for item in sublist], x[1]] for x in lst]
    return final

In [10]:
class MyDataset(Dataset):
    def __init__(self, filename, num_days, debug=False):
        self.data = transform_data(filename, num_days)
        if debug:
          self.data = self.data[:128]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        result = {}
        result["input"] = torch.tensor(self.data[index][0], dtype=torch.float32)
        result["output"] = torch.tensor((self.data[index][1] > 0), dtype=torch.float32)
        
        return result

In [44]:
class Net(nn.Module):
    def __init__(self, num_features):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv1d(num_features, 256, 1)
        self.conv2 = torch.nn.Conv1d(256, 256, 1)
        self.conv3 = torch.nn.Conv1d(256, 128, 1)
        self.conv4 = torch.nn.Conv1d(128, 32, 1)
        self.conv5 = torch.nn.Conv1d(32, 1, 1)
        self.bn1 = nn.BatchNorm1d(256)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)
        self.bn4 = nn.BatchNorm1d(32)
        self.drop = torch.nn.Dropout(0.2)
        self.sigmoid = torch.nn.Sigmoid()
        for net in [
            self.conv1,
            self.conv2,
            self.conv3,
            self.conv4,
            self.conv5,
        ]:
            torch.nn.init.xavier_uniform_(net.weight)

    def forward(self, inputs):
        x = inputs
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.drop(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.drop(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.drop(x)
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.sigmoid(self.conv5(x))
        return x.reshape(-1)

In [48]:
batch_size = 128
epochs = 120
lr = 1e-3  # scheduler will help
train_filename = "10_train.csv"
val_filename = "10_val.csv"
num_days = 7
num_features = len(transform_data(train_filename, num_days)[0][0])

model = Net(num_features).cuda()
loss_fn = nn.BCELoss()
optim = torch.optim.Adam(model.parameters(), lr=lr)

train_data = MyDataset(train_filename, num_days)
train_loader = DataLoader(train_data, batch_size=batch_size)

val_data = MyDataset(val_filename, num_days)
val_loader = DataLoader(train_data, batch_size=batch_size)

loss = 0
for epoch in range(epochs):
    model.train()
    train_loss = 0
    val_loss = 0
    model.train()
    for iteration, data in enumerate(train_loader):
        input = data["input"].cuda()
        gt = data["output"].cuda()
        optim.zero_grad()
        pred = model(input)
        loss = loss_fn(pred, gt)
        loss.backward()
        optim.step()
        train_loss += loss.item()
    model.eval()
    for iteration, data in enumerate(val_loader):
        input = data["input"].cuda()
        gt = data["output"].cuda()
        pred = model(input)
        loss = loss_fn(pred, gt)
        val_loss += loss.item()
    print("Epoch:{}, Training loss: {} Valiation loss: {}".format(epoch, train_loss / len(transform_data(train_filename, num_days)), val_loss / len(transform_data(val_filename, num_days))))

Epoch:0, Training loss: 0.006187545384275044 Valiation loss: 0.021407270828882854
Epoch:1, Training loss: 0.005269108689485467 Valiation loss: 0.01912256598472595
Epoch:2, Training loss: 0.004862690782273626 Valiation loss: 0.017292903363704683
Epoch:3, Training loss: 0.004581338796920214 Valiation loss: 0.015346595148245493
Epoch:4, Training loss: 0.004187897201642748 Valiation loss: 0.013775177796681722
Epoch:5, Training loss: 0.0036344495979515282 Valiation loss: 0.012037714322408041
Epoch:6, Training loss: 0.003215862750811815 Valiation loss: 0.010182179659605026
Epoch:7, Training loss: 0.002805530170648436 Valiation loss: 0.00858843704064687
Epoch:8, Training loss: 0.0024671267891984605 Valiation loss: 0.007130419214566548
Epoch:9, Training loss: 0.0021261360731210793 Valiation loss: 0.005898626024524371


KeyboardInterrupt: ignored

In [ ]:
model.eval()
data = transform_data(val_filename, num_days)
scat = []
for i in range(len(data)):
    input = torch.tensor(data[i][0], dtype=torch.float32).reshape(1,-1,1).cuda()
    gt = torch.tensor(data[i][1] > 0, dtype=torch.float32).cuda()
    pred = model(input)[0]
    scat.append([gt, pred])

In [ ]:
np.mean([(x[0]==(x[1] > 0.5)).item() for x in scat])